In [ ]:
#Intel challenge for scene classification
#Challenge link: https://datahack.analyticsvidhya.com/contest/practice-problem-intel-scene-classification-challe/
#DataSet link: https://www.kaggle.com/nitishabharathi/scene-classification

In [1]:
#Load required packages 
import pandas as pd
import numpy as np
import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os,shutil
from tensorflow.keras import models,layers,optimizers, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

In [2]:
#Reading train data
df= pd.read_csv("train.csv")
#df = df.replace(0,'buildings').replace(1,'forest').replace(2,'glacier').replace(3,'mountain').replace(4,'sea').replace(5,'street')
df = df.replace(0,'Real').replace(1,'Nature').replace(2,'Nature').replace(3,'Nature').replace(4,'Nature').replace(5,'Real')
df.head()

,image_name,label
0,0.jpg,Real
1,1.jpg,Nature
2,2.jpg,Real
3,4.jpg,Real
4,7.jpg,Nature


In [5]:
#Image Augmentation using keras ImageDataGenerator
datagen = ImageDataGenerator(rescale=1.0/255.0,
                           #shear_range = 0.15,
                           #zoom_range = 0.15,
                           horizontal_flip=False,
                           preprocessing_function= preprocess_input,
                           featurewise_std_normalization=True,
                           samplewise_std_normalization=True,
                           fill_mode="constant",
                           validation_split=0.20)
train_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="train/",
                        x_col="image_name",
                        y_col="label",
                        subset="training",
                        batch_size=500,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(228,228))
valid_generator = datagen.flow_from_dataframe(dataframe=df,
                        directory="train/",
                        x_col="image_name",
                        y_col="label",
                        subset="validation", batch_size=100,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(228,228))

C:\Users\RSH\AppData\Local\Programs\Python\Python37\lib\site-packages\keras_preprocessing\image\image_data_generator.py:349: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\RSH\AppData\Local\Programs\Python\Python37\lib\site-packages\keras_preprocessing\image\image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 13628 validated image filenames belonging to 2 classes.
Found 3406 validated image filenames belonging to 2 classes.


In [9]:
#Model construction 
def conv_bn(x):
    x = layers.Conv2D(filters=64, kernel_size=3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    return layers.ReLU()(x)

inputs = layers.Input(shape=(228,228, 3))
x = conv_bn(inputs)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)
x = conv_bn(x)

x = layers.Dropout(0.3)(x)
x = layers.Flatten()(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()
model.compile(loss='binary_crossentropy',optimizer = optimizers.RMSprop(lr=0.01),metrics=['acc'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 228, 228, 3)]     0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 114, 114, 64)      1792      
_________________________________________________________________
batch_normalization_19 (Batc (None, 114, 114, 64)      256       
_________________________________________________________________
re_lu_19 (ReLU)              (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 57, 57, 64)        36928     
_________________________________________________________________
batch_normalization_20 (Batc (None, 57, 57, 64)        256       
_________________________________________________________________
re_lu_20 (ReLU)              (None, 57, 57, 64)        0   

In [10]:
#Train the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
history =model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,verbose=1)

C:\Users\RSH\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
C:\Users\RSH\AppData\Local\Programs\Python\Python37\lib\site-packages\keras_preprocessing\image\image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\RSH\AppData\Local\Programs\Python\Python37\lib\site-packages\keras_preprocessing\image\image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/100
27/27 [==============================] - 465s 17s/step - loss: 0.7785 - acc: 0.6429 - val_loss: 112.8748 - val_acc: 0.6771
Epoch 2/100
27/27 [==============================] - 412s 15s/step - loss: 0.4910 - acc: 0.7580 - val_loss: 171.6062 - val_acc: 0.6765
Epoch 3/100
27/27 [==============================] - 408s 15s/step - loss: 0.3765 - acc: 0.8351 - val_loss: 9.2411 - val_acc: 0.6741
Epoch 4/100
27/27 [==============================] - 414s 15s/step - loss: 0.3036 - acc: 0.8752 - val_loss: 6.1456 - val_acc: 0.4032
Epoch 5/100
27/27 [==============================] - 406s 15s/step - loss: 0.2968 - acc: 0.8724 - val_loss: 2.2558 - val_acc: 0.7826
Epoch 6/100
27/27 [==============================] - 411s 15s/step - loss: 0.1997 - acc: 0.9243 - val_loss: 0.5110 - val_acc: 0.8485
Epoch 7/100
27/27 [==============================] - 406s 15s/step - loss: 0.1748 - acc: 0.9351 - val_loss: 0.7043 - val_acc: 0.8226
Epoch 8/100
27/27 [==============================] - 409s 15s/ste

In [11]:
from tensorflow.keras.models import save_model, load_model
model = model.save_weights("Binary model.h5")